In [113]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar

In [55]:
# input
ETF_data = pd.read_csv('Emerging Asia Pacific ETF List (114).csv')

# filter the inception date
ETF_data['Inception'] = pd.to_datetime(ETF_data['Inception'], format="%d/%m/%Y")
date = datetime.datetime(2016, 1, 1, 0, 0)
ETF_data = ETF_data.loc[ETF_data['Inception'] < date,:].reset_index()

# extract symbol and name of each ETF
symbol_list = ETF_data['Symbol']
name_list = ETF_data['ETF Name']

In [56]:
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))

In [57]:
dic = {'symbol':[],'name':[],'addr':[]}
for i,s in enumerate(symbol_list):
    dic['symbol'].append(s)
    dic['name'].append(name_list[i])
    dic['addr'].append(homepage_list[i])
    print(s,name_list[i],homepage_list[i])
    
# Save this dic
with open("ETF_dic.pkl","wb") as f:
    pickle.dump(dic,f)
# Use
# with open('ETF_dic.pkl','rb') as f:
#     dic = pickle.load(f)

XCEM Columbia EM Core ex-China ETF http://www.columbiathreadneedleetf.com/emergingmarkets/xcem
INCO Columbia India Consumer ETF http://www.columbiathreadneedleetf.com/emergingmarkets/inco
INXX Columbia India Infrastructure Index Fund http://www.columbiathreadneedleetf.com/emergingmarkets/inxx
SCIN Columbia India Small Cap Fund http://www.columbiathreadneedleetf.com/emergingmarkets/scin
AFTY CSOP FTSE China A50 ETF http://www.csopasset.us/en-us/products/etf/a50/
CNHX CSOP MSCI China A International Hedged ETF http://www.csopasset.us/en-us/products/etf/csop-msci-china-a-international-hedged-etf/
YANG Direxion Daily China 3x Bear Shares http://www.direxioninvestments.com/products/direxion-daily-china-bear-3x-etf
YINN Direxion Daily China 3x Bull Shares http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
CHAD Direxion Daily CSI 300 China A Share Bear 1X Shares http://www.direxioninvestments.com/products/direxion-daily-csi-300-china-a-share-bear-1x-shares
CHAU Direx

In [218]:
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - datetime.timedelta(1)
    year = start_date.year
    month = start_date.month
    
    while (temp < end_date):
        temp = temp + datetime.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    return ans

In [219]:
last_day_of_month = get_the_last_day_of_month(datetime.datetime(2016,1,1,0,0),datetime.datetime(2018,12,31,0,0))

In [221]:
# Columbia
def get_Columbia_NAV(symbol,addr,start,last_day_of_month):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    if not os.path.isdir('./data/Columbia'):
        os.mkdir('./data/Columbia')
    
    # crawl    
    if start:
        res = requests.get(addr)
        doc = pq(res.text)
        doc.make_links_absolute(res.url)
        filename = doc('#fundliterature .link-list')('li:nth-child(6)')('a').attr('href')
        filename = filename.replace(' ','%20')
        urllib.request.urlretrieve(filename, "./data/Columbia/columbia.xls")
        print('Download columbia.xls')
        
        # filter the last of the month
        data = pd.read_excel('./data/Columbia/columbia.xls')
        data = data.rename(columns = data.iloc[2]).iloc[3:,:]
        data['Date'] = pd.to_datetime(data['Date'], format="%Y/%m/%d")
        
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        data.to_excel('./data/Columbia/columbia_filter_day.xls')
    
    # get NAV of each symbol from Columbia
    
    data_filter = pd.read_excel('./data/Columbia/columbia_filter_day.xls')
    data_filter = data_filter[data_filter['Fund Symbol'] == symbol]

    data_symbol = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
    data_symbol = data_symbol.merge(data_filter[['Date','NAV']],on='Date',how='left')
    data_symbol.to_excel(f'./data/Columbia/{symbol}.xls')

In [216]:
get_Columbia_NAV(dic['symbol'][0],dic['addr'][0],False,last_day_of_month)